In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb
import warnings 
warnings.filterwarnings('ignore')

In [71]:
diabetes_df = pd.read_csv("diabetes_1.csv")

In [72]:
diabetes_df.head()

,ID,Year,Age,Race,Sex,State,Zip_Code,MSA,Enrollment_Type_Categorized,Enrollment_Months,...,sitagliptin_and_metformin_hydrochloride,Avg_ADI,Max_ADI,Min_ADI,SVI1,SVI3,SVI4,SVI,MDI,Region
0,68802021,2021,68,White,Male,Michigan,48176,11460,10,12.0,...,0,30.157895,67.0,8.0,-27.707059,-18.227551,-27.590337,-27.727856,14.16,NaN
1,3422021,2021,60,Black,Male,Michigan,49224,12980,20,12.0,...,0,80.200000,100.0,52.0,-21.266317,0.272289,-21.271530,-21.282509,15.39,NaN
2,7892020,2020,46,White,Male,Michigan,49016,12980,20,12.0,...,0,82.545455,97.0,63.0,0.529493,0.376685,0.616187,0.555240,15.39,NaN
3,42792020,2020,84,Black,Female,Michigan,49224,12980,10,12.0,...,0,80.200000,100.0,52.0,-21.266317,0.272289,-21.271530,-21.282509,14.16,NaN
4,42792018,2018,82,Black,Female,Michigan,49224,12980,10,12.0,...,0,80.200000,100.0,52.0,-21.266317,0.272289,-21.271530,-21.282509,14.16,NaN


In [73]:
diabetes_df['State'].unique()
state_to_region = {'Northeast': ['New York', 'Massachusetts', 'Pennsylvania', 'Maryland'],
                  'South': ['Florida', 'Texas', 'Tennessee', 'Kentucky', 'North Carolina', 'Virginia', 'Alabama', 'Louisiana', 'Georgia', 'Mississippi', 'Missouri'],
                  'Midwest': ['Illinois', 'Ohio', 'Michigan', 'Wisconsin'],
                  'West': ['Colorado', 'Washington', 'California', 'Nevada', 'Arizona']}
diabetes_df['Region'] = pd.Series()


In [74]:
for i in range(len(diabetes_df['State'])):
    if diabetes_df['State'][i] in state_to_region['Northeast']:
        diabetes_df['Region'][i] = 'Northeast'
    elif diabetes_df['State'][i] in state_to_region['South']:
        diabetes_df['Region'][i] = 'South'
    elif diabetes_df['State'][i] in state_to_region['Midwest']:
        diabetes_df['Region'][i] = 'Midwest'
    else:
        diabetes_df['Region'][i] = 'West'  
        
        
     
         
        
        
        

In [75]:
diabetes_df['Region']

0        Midwest
1        Midwest
2        Midwest
3        Midwest
4        Midwest
          ...   
20406    Midwest
20407    Midwest
20408    Midwest
20409    Midwest
20410    Midwest
Name: Region, Length: 20411, dtype: object